In [124]:
#pyspark packages imported for the required steps.
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import from_utc_timestamp
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
'''
Choosed to import data directly from the link rather than downloading and later uploading so as to save Time."
'''
spark = SparkSession.builder.appName("test").getOrCreate()
url = "http://snap.stanford.edu/data/amazon/productGraph/aggressive_dedup.json.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("multiline","False").json("file://" + SparkFiles.get("aggressive_dedup.json.gz"))
#---Converted "unixReviewTime" "MM-dd-yyyy ---#
df4 = df.select(col("asin"),col("helpful"),col("overall").astype(IntegerType()),col("reviewText"),col("reviewTime"),col("reviewerID"),col("reviewerName"),col("summary"),from_unixtime(col("unixReviewTime"),"MM-dd-yyyy").alias("ChangedReviewTime" ))
#---Used length() to identify the length of review Text---#
df4 = df4.withColumn("length_review", F.length("reviewText"))

df4.createOrReplaceTempView("temptbale")
df3 = spark.sql("select * from temptbale")
#------ Items with the Lease rating ----#
x = df4.agg({"overall": "min"}).collect()[0][0]
df6 = spark.sql("SELECT * FROM temptbale where overall = {}".format(x))
df6.show()
#------ Items with the most rating ----# 
ax = df4.agg({"overall": "max"}).collect()[0][0]
df7 = spark.sql("SELECT * FROM temptbale where overall = {}".format(ax))
df7.show()
#------ Item with the lengthiest review ----# 
lax = df4.agg({"length_review": "max"}).collect()[0][0]
df9 = spark.sql("SELECT * FROM temptbale where length_review = {}".format(lax))
df9.show()

In [132]:
df4.write.parquet("/tmp/Test.parquet")

In [133]:
ls


sample_data/  spark-warehouse/


557
+----------+-------+-------+--------------------+-----------+--------------------+------------+-------+-----------------+-------------+
|      asin|helpful|overall|          reviewText| reviewTime|          reviewerID|reviewerName|summary|ChangedReviewTime|length_review|
+----------+-------+-------+--------------------+-----------+--------------------+------------+-------+-----------------+-------------+
|1492111619| [0, 0]|      5|This book is a gr...|09 26, 2013|A0000440NYTE2D2YB089|  Kristen S.|    Wow|       09-26-2013|          557|
+----------+-------+-------+--------------------+-----------+--------------------+------------+-------+-----------------+-------------+

